In [1]:
import csv
import numpy as np

with open('csv/TCP/evcc_10Mbps_qos_TCP_UDP.csv', newline='') as csvfile:
    #讀取 CSV 檔內容，將每一列轉成一個 dictionary
    rows = csv.DictReader(csvfile)
    lengthlist = []
    timelist = []
    seqlist = []
    tcpseglenlist = []
    infolist = []
    
    for row in rows:
        lengthlist.append(int(row['Length']))
        timelist.append(float(row['Time']))
        seqlist.append(row['Sequence Number'])
        tcpseglenlist.append(row['TCP Segment Len'])
        infolist.append(row['Info'])

In [2]:
round_split_length = []
round_split_time = []
round_split_seq = []
round_split_tcpseglen = []

round_start_point = 0
round_end_point = 0
isfirstround = True

for i in range(1, len(lengthlist)):
    if(lengthlist[i-1] == 72 and lengthlist[i] == 90):
        if(isfirstround):
            isfirstround = False
            continue
        round_end_point = i-1
        if(round_end_point != 0):
            tmp = []
            tmp2 = []
            tmp3 = []
            tmp4 = []
            for j in range(round_start_point, round_end_point):
                tmp.append(lengthlist[j])
                tmp2.append(timelist[j])
                tmp3.append(seqlist[j])
                tmp4.append(tcpseglenlist[j])
            round_split_length.append(tmp)
            round_split_time.append(tmp2)
            round_split_seq.append(tmp3)
            round_split_tcpseglen.append(tmp4)
            
        round_start_point = i-1
        
    if(i == len(lengthlist) - 1):
        round_end_point = i
        tmp = []
        tmp2 = []
        tmp3 = []
        tmp4 = []
        for j in range(round_start_point, round_end_point):
            tmp.append(lengthlist[j])
            tmp2.append(timelist[j])
            tmp3.append(seqlist[j])
            tmp4.append(tcpseglenlist[j])
        round_split_length.append(tmp)
        round_split_time.append(tmp2)
        round_split_seq.append(tmp3)
        round_split_tcpseglen.append(tmp4)

In [3]:
#for debug
print(len(round_split_length))
print(len(round_split_time))
print(len(round_split_seq))
print(len(round_split_tcpseglen))

21
21
21
21


In [4]:
#for debug, check each length
for i in range(0, len(round_split_length)):
    if(len(round_split_length[i]) != len(round_split_time[i])): print("Error")
    if(len(round_split_seq[i]) != len(round_split_time[i])): print("Error")
    if(len(round_split_tcpseglen[i]) != len(round_split_time[i])): print("Error")
    print(len(round_split_length[i]))

102
97
100
95
100
2
99
95
99
95
96
98
101
100
101
96
103
94
99
96
99


In [5]:
#There is 1 time that evcc receives SECC Discovery response almost
#at the same time evcc sends out the next SECC Disocvery request.
#Delete the duplicate SECC Discovery request and the later received SECC Discovery response.
for i in [5]:
    del round_split_length[i]
    del round_split_time[i]
    del round_split_seq[i]
    del round_split_tcpseglen[i]

#for debug
print(len(round_split_length))
print(len(round_split_time))
print(len(round_split_seq))
print(len(round_split_tcpseglen))

#for debug, check each length
for i in range(0, len(round_split_length)):
    if(len(round_split_length[i]) != len(round_split_time[i])): print("Error")
    if(len(round_split_seq[i]) != len(round_split_time[i])): print("Error")
    if(len(round_split_tcpseglen[i]) != len(round_split_time[i])): print("Error")
    print(len(round_split_length[i]))

20
20
20
20
102
97
100
95
100
99
95
99
95
96
98
101
100
101
96
103
94
99
96
99


In [6]:
#for debug, check each length
for i in range(0, len(round_split_length)):
    if(len(round_split_length[i]) != len(round_split_time[i])): print("Error")
    if(len(round_split_seq[i]) != len(round_split_time[i])): print("Error")
    if(len(round_split_tcpseglen[i]) != len(round_split_time[i])): print("Error")
    print(len(round_split_length[i]))

102
97
100
95
100
99
95
99
95
96
98
101
100
101
96
103
94
99
96
99


In [7]:
#preprocessing
for i in range(0, len(round_split_seq)):
    for j in range(0, len(round_split_seq[i])):
        if(round_split_seq[i][j] == ''):
            round_split_seq[i][j] = -1
        else: round_split_seq[i][j] = int(round_split_seq[i][j])
        if(round_split_tcpseglen[i][j] == ''):
            round_split_tcpseglen[i][j] = -1
        else: round_split_tcpseglen[i][j] = int(round_split_tcpseglen[i][j]) 

In [ ]:
#for debug
round_split_seq[2]

In [8]:
time_anchor = []
stage_num = 0 #for debug
retransmission_time_list = []

def check_stage_num(currentlen, time, seq_num, stage_num_valid):
    global stage_num
    global retransmission_time_list
    if(stage_num != stage_num_valid):
        print("Retransmission? " + str(time) + " Seq# " + str(seq_num))
        retransmission_time_list.append(time)
        return False
    else:
        if(stage_num % 2 == 0):
            print("Stage " + str(stage_num//2+1) + " Req catched. Current length = " + str(currentlen+1))
        else: 
            print("Stage " + str((stage_num+1)//2) + " Res catched. Current length = " + str(currentlen+1))
        stage_num = stage_num + 1
        return True
    

for i in range(0, len(round_split_time)): #Test Round i
    print("--------- Test Round " + str(i+1) + " ---------")
    time_anchor_each_round = []

    for j in range(0, len(round_split_time[i])-1):   
        if(j == 0):
            #Start, Stage1(SDP) Start
            if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 0)):
                 time_anchor_each_round.append(round_split_time[i][j])
            
        if(round_split_seq[i][j] == -1 and round_split_seq[i][j+1] == 0):
            #Stage1 (SDP) finish
            if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 1)):
                 time_anchor_each_round.append(round_split_time[i][j])
        
        if(j >= 1 and round_split_seq[i][j-1] == -1 and round_split_seq[i][j] == 0):
            #Stage 2 Start (TCP SYN)
            if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 2)):
                 time_anchor_each_round.append(round_split_time[i][j])
        
        if(round_split_seq[i][j] >= 1655 and round_split_seq[i][j] <= 1670):
            #Stage2 (Session Initialized) finish
            if(round_split_tcpseglen[i][j] == 85 or round_split_tcpseglen[i][j] == 91):
                #SECC "change ciper spec retransmission" + "encrypted handshake message" sent together...
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 3)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 1660~1670, len != 85 nor 91")
                    
        if(round_split_seq[i][j] >= 330 and round_split_seq[i][j] <= 340):
            #Stage3 (SupportedAppProtocol) Start
            if(round_split_tcpseglen[i][j] == 101):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 4)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 330~340, len != 101")
        
        if(round_split_seq[i][j] >= 1745 and round_split_seq[i][j] <= 1755):
            #Stage3 (SupportedAppProtocol) finish
            if(round_split_tcpseglen[i][j] == 69):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 5)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 1745~1755, len != 69")
        
        if(round_split_seq[i][j] >= 435 and round_split_seq[i][j] <= 440):
            #Stage4 (SessionSetup) Start
            if(round_split_tcpseglen[i][j] == 85):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 6)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 435~440, len != 85")
        
        if(round_split_seq[i][j] >= 1815 and round_split_seq[i][j] <= 1825):
            #Stage4 (SessionSetup) finish
            if(round_split_tcpseglen[i][j] == 101):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 7)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 1815~1825, len != 101")
        
        if(round_split_seq[i][j] >= 520 and round_split_seq[i][j] <= 555):
            #Stage5 (ServiceDiscovery) Start
            if(round_split_tcpseglen[i][j] == 85):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 8)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 520~555, len != 85")
        
        if(round_split_seq[i][j] >= 1915 and round_split_seq[i][j] <= 1925):
            #Stage5 (ServiceDiscovery) finish
            if(round_split_tcpseglen[i][j] == 165):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 9)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 1915~1925, len != 165")
        
        if(round_split_seq[i][j] >= 605 and round_split_seq[i][j] <= 610):
            #Stage6 (ServicePaymentSelection) Start
            if(round_split_tcpseglen[i][j] == 85):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 10)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 605~610, len != 85")
        
        if(round_split_seq[i][j] >= 2075 and round_split_seq[i][j] <= 2085):
            #Stage6 (ServicePaymentSelection) finish
            if(round_split_tcpseglen[i][j] == 85):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 11)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 2075~2085, len != 85")
        if(round_split_seq[i][j] >= 2115 and round_split_seq[i][j] <= 2125):
            #Stage7 (ServicePaymentSelection) Start
            if(round_split_tcpseglen[i][j] == 97):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 12)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 2115~2125, len != 97")
        
        if(round_split_seq[i][j] >= 2165 and round_split_seq[i][j] <= 2175):
            #Stage7 (PaymentDetail) finish
            if(round_split_tcpseglen[i][j] == 101):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 13)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 2165~2175, len != 101")
        
        if(round_split_seq[i][j] >= 2215 and round_split_seq[i][j] <= 2220):
            #Stage8 (Authorization) Start
            if(round_split_tcpseglen[i][j] == 373):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 14)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 2215~2220, len != 373")
        
        if(round_split_seq[i][j] >= 2265 and round_split_seq[i][j] <= 2275):
            #Stage8 (Authorization) finish
            if(round_split_tcpseglen[i][j] == 85):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 15)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 2265~2275, len != 85")
        
        if(round_split_seq[i][j] >= 2585 and round_split_seq[i][j] <= 2595):
            #Stage9 (ChargeParameterDiscovery) Start
            if(round_split_tcpseglen[i][j] == 101):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 16)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 2585~2595, len != 101")
        
        if(round_split_seq[i][j] >= 2350 and round_split_seq[i][j] <= 2360):
            #Stage9 (ChargeParameterDiscovery) finish
            if(round_split_tcpseglen[i][j] == 405):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 17)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 2350~2360, len != 405")
        
        if(round_split_seq[i][j] == 2691):
            #Stage10 (PowerDelivery) Start
            if(round_split_tcpseglen[i][j] == 85):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 18)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 2690~2695, len != 85")
        
        if(round_split_seq[i][j] >= 2755 and round_split_seq[i][j] <= 2765):
            #Stage10 (PowerDelivery) finish
            if(round_split_tcpseglen[i][j] == 85):
                if(check_stage_num(len(time_anchor_each_round), round_split_time[i][j], round_split_seq[i][j], 19)):
                    time_anchor_each_round.append(round_split_time[i][j])
            else:
                if(round_split_tcpseglen[i][j] != 0):
                    print("len = " + str(round_split_tcpseglen[i][j]))
                    print("Seq# 2755~2765, len != 85")
    
    stage_num = 0
    time_anchor.append(time_anchor_each_round)
    
print(retransmission_time_list)

--------- Test Round 1 ---------
Stage 1 Req catched. Current length = 1
Stage 1 Res catched. Current length = 2
Stage 2 Req catched. Current length = 3
len = 6
Seq# 1660~1670, len != 85 nor 91
Stage 2 Res catched. Current length = 4
Stage 3 Req catched. Current length = 5
Retransmission? 6.329786558 Seq# 336
Stage 3 Res catched. Current length = 6
Stage 4 Req catched. Current length = 7
Retransmission? 6.485797617 Seq# 1748
Stage 4 Res catched. Current length = 8
Stage 5 Req catched. Current length = 9
Retransmission? 7.036121633 Seq# 1817
Stage 5 Res catched. Current length = 10
Stage 6 Req catched. Current length = 11
Stage 6 Res catched. Current length = 12
Stage 7 Req catched. Current length = 13
Stage 7 Res catched. Current length = 14
Stage 8 Req catched. Current length = 15
Stage 8 Res catched. Current length = 16
Stage 9 Req catched. Current length = 17
Stage 9 Res catched. Current length = 18
Stage 10 Req catched. Current length = 19
Stage 10 Res catched. Current length = 20


In [9]:
#Verify retransmission packet info
for retrans_time in retransmission_time_list:
    print(infolist[timelist.index(retrans_time)])

[TCP Retransmission] 33032  >  52385 [PSH, ACK] Seq=336 Ack=1748 Win=64128 Len=101 TSval=1600944033 TSecr=3817204048
[TCP Spurious Retransmission] 52385  >  33032 [PSH, ACK] Seq=1748 Ack=437 Win=131072 Len=69 TSval=3817204575 TSecr=1600943710
[TCP Spurious Retransmission] 52385  >  33032 [PSH, ACK] Seq=1817 Ack=522 Win=131072 Len=101 TSval=3817205012 TSecr=1600944190
[TCP Retransmission] 33036  >  52385 [PSH, ACK] Seq=2217 Ack=2271 Win=64128 Len=373 TSval=1600991462 TSecr=4233178419
[TCP Retransmission] 33042  >  52385 [PSH, ACK] Seq=607 Ack=2084 Win=64128 Len=85 TSval=1601057539 TSecr=899172598
[TCP Spurious Retransmission] 52385  >  33042 [PSH, ACK] Seq=2084 Ack=692 Win=131072 Len=85 TSval=899173004 TSecr=1601057111
[TCP Spurious Retransmission] 52385  >  33056 [PSH, ACK] Seq=1919 Ack=607 Win=131072 Len=165 TSval=283914223 TSecr=1601216786
[TCP Retransmission] 33056  >  52385 [PSH, ACK] Seq=2120 Ack=2169 Win=64128 Len=97 TSval=1601217622 TSecr=283914280
[TCP Retransmission] 33060  > 

In [10]:
#for debug
print(len(time_anchor))
for i in range(0, len(time_anchor)):
    print("i = " + str(i) + " , len = " + str(len(time_anchor[i])))

20
i = 0 , len = 20
i = 1 , len = 20
i = 2 , len = 20
i = 3 , len = 20
i = 4 , len = 20
i = 5 , len = 20
i = 6 , len = 20
i = 7 , len = 20
i = 8 , len = 20
i = 9 , len = 20
i = 10 , len = 20
i = 11 , len = 20
i = 12 , len = 20
i = 13 , len = 20
i = 14 , len = 20
i = 15 , len = 20
i = 16 , len = 20
i = 17 , len = 20
i = 18 , len = 20
i = 19 , len = 20


In [10]:
'''
#TIMEOUT Dealing. TIMEOUT ROUND is NOT considered
del time_anchor[10]
del time_anchor[9]
del time_anchor[6]
'''

In [11]:
#for debug
print(len(time_anchor))
for i in range(0, len(time_anchor)):
    print("i = " + str(i) + " , len = " + str(len(time_anchor[i])))

20
i = 0 , len = 20
i = 1 , len = 20
i = 2 , len = 20
i = 3 , len = 20
i = 4 , len = 20
i = 5 , len = 20
i = 6 , len = 20
i = 7 , len = 20
i = 8 , len = 20
i = 9 , len = 20
i = 10 , len = 20
i = 11 , len = 20
i = 12 , len = 20
i = 13 , len = 20
i = 14 , len = 20
i = 15 , len = 20
i = 16 , len = 20
i = 17 , len = 20
i = 18 , len = 20
i = 19 , len = 20


In [12]:
time_anchor[0]

[4.853754477,
 5.055536226,
 5.402245812,
 5.976198695,
 6.005919987,
 6.37272989,
 6.437281907,
 6.999961573,
 7.004773858,
 7.0851573,
 7.142325245,
 7.428109298,
 7.549735105,
 7.664765013,
 7.732307793,
 7.95884355,
 7.964602983,
 8.283250027,
 8.333360374,
 8.38756814]

In [13]:
path = 'time_record/qos_10Mbps_TCP_UDP.txt'
f = open(path, 'w')
for i in range(0, len(time_anchor)):
    for j in range(1, len(time_anchor[i]), 2):
        print((time_anchor[i][j] - time_anchor[i][j-1])*1000, file=f, end="")
        f.writelines('\t')
    f.writelines('\t')
    print((time_anchor[i][3] - time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    print((time_anchor[i][len(time_anchor[i])-1] - time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    f.writelines('20')
    f.writelines('\t')
    f.writelines('0')
    f.writelines('\t')
    f.writelines('\n')
f.close()